In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

13.	Driver-Specific Track Struggles:<br>
○	Identify circuits where specific drivers consistently struggle or excel.


In [3]:
import pandas as pd
results = pd.read_csv("final_dataset/results.csv")
races = pd.read_csv("final_dataset/races.csv")
circuits = pd.read_csv("final_dataset/circuits.csv")
drivers = pd.read_csv("final_dataset/drivers.csv")

results = results.merge(drivers[['driverId', 'forename','surname']], on='driverId', how='left')
results['forename'] = results['forename'].astype('str')
results['surname'] = results['surname'].astype('str')
results['Name']  = results['forename']+" "+results['surname']
results.drop(columns=['forename','surname'],inplace=True)
results = results.merge(races[['raceId', 'circuitId']], on='raceId', how='left')
results = results.merge(circuits[['circuitId','name']],on=['circuitId'],how='left')
results.rename(columns={'name':'circuitname'},inplace=True)

Overall struggling

In [ ]:
driver_circuit_stats = results.groupby(['Name', 'circuitname']).agg(
    avg_position=('positionOrder', 'mean'),  
    avg_points=('points', 'mean'),  
    dnf_rate=('status', lambda x: (x != 'Finished').mean())  
).reset_index()

print("Driver Performance by Circuit:")
print(driver_circuit_stats.head())


Driver Performance by Circuit:
           Name                     circuitname  avg_position  avg_points  \
0  adolf brudes                     nürburgring     16.000000    0.000000   
1   adolfo cruz   autódromo juan y oscar gálvez     16.000000    0.000000   
2  adrian sutil  albert park grand prix circuit     12.428571    1.142857   
3  adrian sutil    autodromo nazionale di monza     15.428571    0.714286   
4  adrian sutil      autódromo josé carlos pace     13.714286    1.142857   

   dnf_rate  
0  1.000000  
1  1.000000  
2  0.714286  
3  0.857143  
4  1.000000  


From these we can say that adolf brudes struggles the most in <b>nürburgring</b> concluded based on large avg position ,low average position and high DNF rate

Driver Specific

In [5]:
driver = 'max verstappen'

driver_data = driver_circuit_stats[driver_circuit_stats['Name'] == driver]
struggling_circuits = driver_data.sort_values(by=['dnf_rate', 'avg_points'], ascending=[False, True]).head(3)
best_circuits = driver_data.sort_values(by=['avg_position', 'avg_points'], ascending=[True, False]).head(3)

In [6]:
print("\nCircuits Where ",driver," Struggles: ")
print(struggling_circuits)

print("\nCircuits Where ",driver," excels: ")
print(best_circuits)


Circuits Where  max verstappen  Struggles: 
                Name                           circuitname  avg_position  \
5065  max verstappen  autodromo internazionale del mugello     19.000000   
5070  max verstappen         bahrain international circuit      9.909091   
5066  max verstappen          autodromo nazionale di monza      8.500000   

      avg_points  dnf_rate  
5065    0.000000  1.000000  
5070    9.818182  0.454545  
5066    7.900000  0.400000  

Circuits Where  max verstappen  excels: 
                Name                    circuitname  avg_position  avg_points  \
5077  max verstappen         circuit park zandvoort      1.250000   23.500000   
5084  max verstappen   losail international circuit      1.333333   23.333333   
5086  max verstappen  miami international autodrome      1.333333   23.333333   

      dnf_rate  
5077       0.0  
5084       0.0  
5086       0.0  


From these we can conclude that max verstappen <b>struggles</b> most in <b> autodromo internazionale del mugello </b> circuit<br>
Also he is known to <b>excel</b> most  in <b> circuit park zandvoort </b> circuit

14.	Championship Retention Probability:<br>
○	What is the probability that this season’s winner will retain the title in the next season? Analyze historical trends of back-to-back champions.
